# Image restoration

In general, every degradation and restoration of an image is modeled as:

$$ \hat{f}(x,y) = \left[ f(x,y) * h(x,y) + \eta(x,y) \right] * r(x,y) $$

where the convolution with $h(x,y)$ represents the degradation function $H(\mu,\sigma)$ (as an LTI system or whatever it is called in image), $\eta(x,y)$ is the additive noise and $r(x,y)$ encompases the restoration filters (another set of LTI systems). The output of the degradation process (upto $r(x,y)$) is $g(x,y)$, which is called the "observation".

## Noise

### Types of noise

There are several types of noise:
1. **Gaussian**: Additive White Gaussian Noise

    There is no real system that produces Gaussian noise. Still, it is the most used noise model. Partly, because mathematically it is one of the most easy to work with. It is also a good approximation to other types of noise, e.g. when looking at one pixel or group of pixels.
1. **Rayleigh**

    As opposed to AWGN, it does appear in physical systems. The formula is:

    $$ p(z) = \left\{ \begin{array}{l} \frac{2(z-a)}{b} \, e^{-\frac{(z-a)^2}{b}} \quad \textrm{if} \, z \geq a, \\ 0 \quad \textrm{if} \, z \leq a \end{array} \right. $$

    where $b$ and $a$ follow:

    $$ \bar{z} = a + \sqrt{\frac{\pi b}{4}} $$

    $$ \sigma^2 = \frac{b(4-\pi)}{4} $$

    This noise is used to model the ones that appear, for instance, in magnetic resonance imaging, underwater imaging or in camera sensors. For many real cases it is a very good model.

1. **Gamma**
    
    It follows the expression:

    $$ p(z) = \frac{a(b-1)^{b-1}}{(b-1)!}e^{-(b-1)} $$

1. **Exponential**

    This type of noise shows no noise for negative values of the independent variable.

    $$ p(z) = \left\{ \begin{array}{l} a \cdot e^{-az} \quad \textrm{if} \, z \geq 0 \\ 0 \quad \textrm{if} \, z \leq 0 \end{array} \right. $$

    where $\bar{z} = 1/a$ and $\sigma^2 = 1/a^2$.

    It is used sometimes as a model for quantization noise. It is also used as a model for the error in predictive coding.

1. **Uniform**

    The noise has the exact same amplitude for all values in the range $[a,b]$ and has amplitude $p(z) = \frac{1}{b-a}$ in that interval, while its amplitude is 0 outside of it.
    
    This is a good model for **quantization noise**.

1. **Impulse**: salt and pepper

    It has a small set of values. It is represented by a few peaks at them in the PDF graph.


### Noise estimation

The type of noise can sometimes even be recognized by looking at the histogram of the image. When we add noise to the image, the resulting PDF is equal to the convolution of the image PDF with the noise PDF.

The type of noise and its parameters can actually be estimated from the histogram. First, a region as flat as possible in the image is selected, in order to optimally get a pure peak and the noise surrounding it. The region needs to be big enough to get a histogram that is not too noisy. Then, by computing the mean and variance of that region, we can obtain the different parameters of the different types of noise, which are usually related to the first two. We can fit the best possible model for each of the different types of noise: the one with the smallest divergence would be the correct one.

It is also normal to estimate the type of noise in several regions to check if the noise is the same or in order to get many estimations of the same one, which would yield a better estimate.

## Filtering

The different types of noise are more or less effectively filtered by different types of filters. For example, salt and pepper noise is grately reduced by the meadian filter; Gaussian noise by non-local means averaging. So, knowing the type of noise present helps us decide which operations we need to perform in order to reduce it.

It is useful to check after filtering that the difference between the original noisy image and the filtered one gives an error that corresponds with the type of noisy that was being considered and modeled.

By transforming the degradation part of the first equation by Fourier Transform:

$$ \hat{f}(x,y) = f(x,y) * h(x,y) $$
$$ G(u,v) = F(u,v) \cdot H(u,v) $$

And dividing both sides by the degradation filter $H(u,v)$:

$$ F(u,v) = \frac{G(u,v)}{H(u,v)} $$

This is called inverse filtering, but it is not always possible due to the invertibility of $H(u,v)$ (and it can be zero).

An impulse (delta) function can be used to calibrate the restoration filters, because it gives both the original $f(x,y)$ image and the degraded $g(x,y)$, so we only need to obtain (despejar) $h(x,y)$.

### Wiener filter

Recalling the degradation model, the Wiener filter will aim to minimize the MSE between the original image and the degraded one:

$$ e^2 = E[f(x,y) - \hat{f}(x,y)] $$

As the original image is unknown, the expected value is used to take advantage of a statistical characterization of it.

The expression of the filtering process in Fourier domain is:

$$ \hat{F}(u,v) = \frac{H^*(u,v)}{H^2(u,v)+S_\eta/S_f} \cdot G(u,v) $$

where the first multiplicand in the right hand side is the Wiener filter, built to minimize the MSE, $S_\eta$ and $S_f$ are respectively the power spectra of the noise and signal.

A power spectrum is a Fourier Transform of the autocorrelation of a variable. We do not need the exact values of its input but some statistical characterization of it. Because it is hard to analyze both the input image and noise, their power spectra division is replaced by a constant (which needs to be estimated).